In [ ]:
using ControlSystems
using LinearAlgebra
using Plots
using Interact
using Printf

# Bioreaktor mit PID-Regler

Zustandsraumdarstellung des Bioreaktors von Übungsblatt 5:

In [ ]:
A = [-0.1 0.008
     -0.6 0.0  ]
b = [-0.01, 0.1]
c = [1 0]
d = 0
plant = ss(A, b, c, d)

In [ ]:
plot(step(plant), label="Sprungantwort der Strecke", legend=:bottomright)

PID-Regler nach Ziegler-Nichols (s. Übung 5):

[Transfer function (Übertragungsfunktion) ist eine Repräsentation von LTI-Systemen im Frequenzbereich (kein Gegenstand von Regelungstechnik und Systemdynamik)]

In [ ]:
kp = 6
T_I = 27.778
T_D = 6.667

controller = pid(kp=kp, ki=T_I, kd=T_D, time=true, series=true)

In [ ]:
# Multipliziere System und Regler für Hintereinanderschaltung
controlled_sys = tf(plant) * controller
# Berechne Sprungantwort des geschlossenen Regelkreises
pid_solution = step(feedback(controlled_sys))
plot(pid_solution, label="PID Regler", legend=:bottomright)
plot!(t -> 1, label="σ(t)")

# Aufgabe 1

Der Sensor (PT1-Glied) wird in Zustandsraumdarstellung definiert.

In [ ]:
T = 1 / log(√2)
# Wahre statische Verstärkung des Sensors
K = 5
sensor = ss(-1/T, K/T,1,0)

Sprungantwort des Sensors (PT1-Verhalten):

In [ ]:
plot(step(sensor), xlim=(0, 10), ylim=(0, 7), label="Sensor")
plot!(t -> K, label=false)
plot!(t -> K/T * t, label=false)

Der geschlossene Regelkreises (jetzt inklusive PT1-Messelement) ist instabil:

In [ ]:
measured_sys = controlled_sys * sensor
plot(step(feedback(measured_sys), 100), label="Mit PT1 Messung")

## Aufgabe 1.1

Messwerte der Sprungantwort:

In [ ]:
t̂ = [2, 4, 6]

In [ ]:
v̂ = [2.5, 3.75, 4]

In [ ]:
@manipulate for K_ = 0:0.01:8
    p = scatter(t̂, v̂, label="Messungen", xlim=(0, 10), ylim=(0,5))
    # PT1 Sprungantwort
    g_σ(t) = K_ * (1 - exp(-t / T))
    # Werte ohne Messunsicherheit
    v = g_σ.(t̂)
    plot!(g_σ, label="PT1 Sprungantwort", c=:green)
    plot!([t̂ t̂]',[v̂ v]', label=["Residuum" false false], c=:red)
    ê = v̂ - v
    hbox(@sprintf("Residuum |ê| = %.2f", norm(ê)), hskip(5em), p)
end

Aufstellen der Messmatrix aus PT1-Sprungantwort:

In [ ]:
H = 1 .- exp.(-t̂ / T)

## Aufgabe 1.2

Bestimme die Pseudoinverse von H:

In [ ]:
# H' is die transponierte von H
H_pinv = inv(H' * H) * H'

Lösen des least squares Problems nach Parameter $\hat{K}$:

In [ ]:
K̂ = H_pinv * v̂

Der backslash-operator '\\' kann genutzt werden um das Gleichungssystem zu lösen anstatt die Inverse zu berechnen:

In [ ]:
(H' * H) \ (H' * v̂)

Der backslash-operator löst auch direkt das least squares Problem:

In [ ]:
H \ v̂

## Aufgabe 1.3

Unsicherheit der Messungen

In [ ]:
σ = √0.3

Unsicherheit der Schätzung von $\hat{K}$

In [ ]:
σ_K = √(σ^2 * inv(H' * H))

## Aufgabe 1.4

Residuum:

In [ ]:
ê = v̂ - H * K̂

Teste Modellgüte durch Vergleich von Residuennorm mit erwarteten Residuen

In [ ]:
n, m = length(v̂), length(K̂)

Quadrierte Residuennorm

In [ ]:
residual_norm = dot(ê, ê)

Erwartete Residuen

In [ ]:
expected_residual_norm = (n - m) * σ^2

In [ ]:
residual_norm < 10 * expected_residual_norm ? "Modellgüte ausreichend!" : "Modellgüte nicht ausreichend!"

## Kompensation der Zeitverzögerung

Ist ein Schätzwert für die Verstärkung $\hat{K}$ bekannt, kann eine proportionale Kompensation genutzt werden:

In [ ]:
compensation = 1 / K̂
plot(step(feedback(measured_sys * compensation), 150), label="Mit P Kompensation", legend=:bottomright)
plot!(pid_solution, label="Perfekte Messung")
plot!(t -> 1, label="σ(t)")

Ist darüberhinaus die Zeitkonstante bekannt, kann ein PDT1-Glied zusätzlich die Zeitkonstante verringern. Die Kompensation genügt der DGL $$T_c\dot{y} + y = \frac{1}{K} (T\dot{u} + u)$$ und verringert die Zeitkonstante von $T$ zu $T_c$.

In [ ]:
T_c = 0.1 * T
# Definition des PDT1-Glieds im Frequenzbereich als Übertragungsfunktion (nicht Teil der RtSys Vorlesung), äquivalant zu obiger DGL
compensation = 1/K̂ * tf([T,1],[T_c,1])
# Transformation in Zustandsraumdarstellung
ss(compensation)

In [ ]:
@manipulate for k_c = 0.01:0.01:1
    T_c = k_c * T
    compensation = 1/K̂ * tf([T,1],[T_c,1])
    plot(step(feedback(measured_sys * compensation), 150), label="Mit Kompensation", legend=:bottomright, ylim=[-0.8, 1.5])
    plot!(pid_solution, label="Perfekte Messung")
    plot!(t -> 1, label="σ(t)")
end

# Aufgabe 2

In [ ]:
t̂ = [1, 2, 3, 4, 5]

In [ ]:
v̂ = [2, 4, 2, 5, 1]

In [ ]:
scatter(t̂, v̂, label="Messungen", legend=:topleft)

In [ ]:
γ(t) = t == 0

## Aufgabe 2.1

Definiere Messmatrix $H$ mit Parametern Nutzsignal $u$ und Messfehler $h_1$

In [ ]:
c1 = ones(length(t̂))

In [ ]:
c2 = γ.(t̂ .- 2)

In [ ]:
H1 = [c1 c2]

Löse least squares Problem

In [ ]:
û, ĥ1 = H1 \ v̂

In [ ]:
plot!(1:0.01:5, t -> û + ĥ1 * γ(t-2), label="Least squares mit p1")

## Aufgabe 2.2

Definiere Messmatrix $H$ mit Parametern Nutzsignal $u$ und Messfehlern $h_1, h_2$

In [ ]:
c3 = γ.(t̂ .- 4)

In [ ]:
H2 = [H1 c3]

Löse least squares Problem

In [ ]:
û, ĥ1, ĥ2 = H2 \ v̂

In [ ]:
plot!(1:0.01:5, t -> û + ĥ1 * γ(t-2) + ĥ2 * γ(t-4), label="Least squares mit p1,p2")

Der Messfehler bei $t=4$ hat die least squares Lösung stark beeinflusst.